In [10]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
tf.test.gpu_device_name()

## Imports

In [37]:
import pandas as pd
import numpy as np
from tensorflow import keras

In [13]:
target_language = "bn"
PATH_template = "/kaggle/input/dakshina/dakshina_dataset_v1.0/{}/lexicons/{}.translit.sampled.{}.tsv"
train_path = PATH_template.format(target_language, target_language, "train")
dev_path = PATH_template.format(target_language, target_language, "dev")
test_path = PATH_template.format(target_language, target_language, "test")

train_data = pd.read_csv(train_path, sep = '\t', header = None)
dev_data = pd.read_csv(dev_path, sep = '\t', header = None)
test_data = pd.read_csv(test_path, sep = '\t', header = None)

In [17]:
train_data.head()

In [18]:
#word wise list
train_input = [str(w) for w in train_data[1]]
train_target = ["\t" + str(w) + "\n" for w in train_data[0]]

dev_input = [str(w) for w in dev_data[1]]
dev_target = ["\t" + str(w) + "\n" for w in dev_data[0]]

test_input = [str(w) for w in train_data[1]]
test_target = ["\t" + str(w) + "\n" for w in train_data[0]]

In [22]:
input_characters = set()
target_characters = set()
input_characters.add(' ')
target_characters.add(' ')

for word in train_input:
    for char in word:
        input_characters.add(char)
for word in dev_input:
    for char in word:
        input_characters.add(char)
for word in test_input:
    for char in word:
        input_characters.add(char)

for word in train_target:
    for char in word:
        target_characters.add(char)
for word in dev_target:
    for char in word:
        target_characters.add(char)
for word in test_target:
    for char in word:
        target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in train_input])
max_decoder_seq_length = max([len(txt) for txt in train_target])

print("Number of samples:", len(train_input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

In [25]:
print(input_characters)
print(target_characters)

In [26]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [29]:
print(input_token_index)
print(target_token_index)

In [ ]:
input_texts = train_input
target_texts = train_target

In [31]:
encoder_input_data = np.zeros((len(train_input), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(train_input), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(train_input), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [32]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

In [36]:
for i, (input_text, target_text) in enumerate(zip(train_input, train_target)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0  
    encoder_input_data[i, t + 1 :, input_token_index[' ']] = 1.0 
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[' ']] = 1.0
    decoder_target_data[i, t:, target_token_index[' '] ] = 1.0 

In [38]:
batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = len(train_input)  # Number of samples to train on.

In [39]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [40]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")
